# Cost Report

In [ ]:
# Bigquery Cost Table ID is Required 
# "ProjectID.DatasetID.TableID"
# Example: "my_project_id.my_dataset_billing_dump.gcp_billing_export_v1_000000_000000_000000"
input_bq_cost_table_id: str = ""


## Import packages

In [ ]:
!pip install --upgrade pip
!pip uninstall -y cromonitor
!pip install --no-cache-dir git+https://github.com/broadinstitute/cromwell-task-monitor-bq-vis.git

In [ ]:
import os
import pandas as pd
pd.set_option('display.max_rows', 200) # so we can see all df rows

from cromonitor.query.cost import CostQuery
from cromonitor.plotting.cost_plots import CostPlots
from cromonitor.table import utils as tableUtils
from cromonitor.jupyter import utils as jupyterUtils
from cromonitor.fiss import utils as fissUtils 


In [ ]:
workspace_namespace = os.environ["WORKSPACE_NAMESPACE"]
workspace_name = os.environ["WORKSPACE_NAME"]

## Select Submission and Workflow 

In [ ]:
###
# Get a list of submission for the workspace
###
submissions_df_sorted = fissUtils.get_list_of_submissions(
    workspace_namespace = workspace_namespace,
        workspace_name= workspace_name,
)
display(submissions_df_sorted)
print("Select Submission ID Below")
###
# Select Submission Id
###
sumission_ids = submissions_df_sorted['submissionId'].tolist()
selected_submission_id = jupyterUtils.create_submission_selector(
options = sumission_ids
)

In [ ]:
####
# Get all the workflow ids assoccaited with the submission
####
workflow_id_df_sorted = fissUtils.get_submission_workflow_ids(
        workspace_namespace=workspace_namespace,
        workspace_name=workspace_name,
        submission_id=selected_submission_id.value,
)
display(workflow_id_df_sorted)
print("Select Workflow ID Below")
###
# Select Workflow Id 
###
workflow_ids = workflow_id_df_sorted['workflowId'].tolist()
selected_workflow_id = jupyterUtils.create_workflow_selector(
options = workflow_ids
)

In [ ]:
####
# Create a workflow object to retrieve and store the selected workflow information
####
selected_workflow_info = fissUtils.Workflow(
            workspace_namespace=workspace_namespace,
            workspace_name=workspace_name,
            submission_id=selected_submission_id.value,
            parent_workflow_id=selected_workflow_id.value,
)

## Query BQ database

In [ ]:
cost_query_job = CostQuery(
        workflow_id=selected_workflow_info.parent_workflow_id, 
        bq_cost_table=input_bq_cost_table_id,
        start_time=selected_workflow_info.workflow_start_time,
        end_time=selected_workflow_info.workflow_end_time,
)

In [ ]:
# How much will it cost to run a query?
cost_query_job.get_cost_to_query()

In [ ]:
# The logic below will do one of two things. 
# 1. Query the cost of the workflow and save it locally as a pkl file,
# 2. If it finds a pkl file in the current directory with the same workflow id it will load that file
# as a pd dataframe. 

cost_report_filename = os.path.join(cost_query_job.workflow_id + '_cost_report.pkl')
if os.path.exists(cost_report_filename):
    print("Loading data from local files")
    cost_df = tableUtils.load_dataframe(cost_report_filename)
else:
    print("Loading data from querying BQ database")
    cost_query_job.query_cost()
    cost_df = cost_query_job.results(to_dataframe=True)
    ## Saves dataframe locally in pickle format
    if not cost_df.empty :
        cost_df.to_pickle(cost_report_filename)
    else:
        print("Empty Database: No Files Saved")

In [ ]:
cost_df

## Plot Cost

In [ ]:
cost_plot_obj = CostPlots(cost_df)

In [ ]:
cost_plot_obj.plot_workflow_cost(group_by_description=False)

In [ ]:
####
# Select task to plot
####
#Get an array of task names in workflow
AllTaskNames = cost_df.task_name.unique()
# Create the SelectMultiple widget
task_selector = jupyterUtils.create_task_selector(AllTaskNames)

In [ ]:
cost_plot_obj.plot_task_cost(task_name=task_selector.value[0], group_by_description=True)